In [1]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 520.0 kB/s eta 0:00:00


In [2]:
from transformers  import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show


In [3]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

Output hidden; open in https://colab.research.google.com to view.

# Scaled dot-product attention

In [38]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
print(inputs)
inputs.input_ids

{'input_ids': tensor([[ 2051, 10029,  2066,  2019,  8612]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [39]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [40]:
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [41]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size() # [batch_size, seq_len, hidden_dim],

torch.Size([1, 5, 768])

In [42]:
# QKV
import torch
from math import sqrt
import torch.nn.functional as F
query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k) # Batch Matrix multiplication

print(scores.size())
weights = F.softmax(scores, dim=-1)
print(weights.sum(dim=1))

attn_scores = torch.bmm(weights, value)
print(attn_scores.size())

torch.Size([1, 5, 5])
tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)
torch.Size([1, 5, 768])


In [43]:
def scaled_dot_product_attention(query, key, value):
    dim_k = key.size(-1)
    socres = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

# Multihead Attention

In [97]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim) -> None:
        super().__init__()
        self.query = nn.Linear(embed_dim, head_dim)
        self.key = nn.Linear(embed_dim, head_dim)
        self.value = nn.Linear(embed_dim, head_dim)

    def forward(self, query, key, value, mask=None):
        return scaled_dot_product_attention(
            self.query(query), self.key(key), self.value(value), mask
        )


In [69]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads

        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, mask=None):
        x = torch.cat([h(query, key, value, mask)for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x



In [71]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds, inputs_embeds, inputs_embeds)
attn_output.size()

torch.Size([1, 5, 768])

In [47]:
from bertviz import head_view
from transformers import AutoModel
model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)
sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"
viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])
head_view(attention, tokens, sentence_b_start, heads=[8])

Output hidden; open in https://colab.research.google.com to view.

# Feed Forward Layer


In [72]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.gelu(self.linear1(x))
        x = self.linear2(x)
        x = self.dropout(x)
        return x

In [73]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_output)
ff_outputs.size()

torch.Size([1, 5, 768])

# Adding Layer Normalization

In [78]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = MultiHeadAttention(config)
        self.layernorm1 = nn.LayerNorm(config.hidden_size)
        self.layernorm2 = nn.LayerNorm(config.hidden_size)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        hidden_state = self.layernorm1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state, hidden_state, hidden_state)
        # Apply feed-forward layer with a skip connection
        x = x + feed_forward(self.layernorm2(x))
        return x


In [79]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

# Positional Embeddings

In [80]:
class Embedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0) # [] -> [[]]
        token_embedding = self.token_embedding(input_ids)
        position_embedding = self.position_embedding(position_ids)
        embedding = token_embedding + position_embedding
        embedding = self.layer_norm(embedding)
        embedding = self.dropout(embedding)
        return embedding


In [81]:
embedding_layer = Embedding(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

# Transformer Encoder

In [82]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embedding(config)
        self.encoder_layers = nn.ModuleList(
            [TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)]
        )

    def forward(self, x):
        x = self.embeddings(x)
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x



In [83]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

# Adding a Classification Head

In [84]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, x):
        x = self.encoder(x)
        x = self.dropout(x[:, 0, :])# select hidden state of [CLS] token
        x = self.classifier(x)
        return x


In [85]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 3])

# Decoder

In [28]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [36]:
scores.masked_fill(mask==0, -float("inf"))

tensor([[[26.6275,    -inf,    -inf,    -inf,    -inf],
         [-0.5488, 29.3876,    -inf,    -inf,    -inf],
         [-0.7076,  0.9663, 26.9218,    -inf,    -inf],
         [-2.1124, -1.1041,  0.6087, 28.3915,    -inf],
         [-1.9693, -0.3208, -0.8279, -0.3535, 28.5138]]],
       grad_fn=<MaskedFillBackward0>)

In [104]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = key.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k) # Batch Matrix multiplication
    if mask is not None:
        scores = scores.masked_fill(mask==0, -float("inf"))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

In [108]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.mask_attention = MultiHeadAttention(config)
        self.self_attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self,x, encoder_output, dec_mask, memory_mask):
        x = self.mask_attention(x, x, x, dec_mask)
        x = self.self_attention(x, encoder_output, encoder_output, memory_mask)
        x = self.feed_forward(x)
        return x




In [111]:
memory_mask = torch.ones(seq_len, seq_len).unsqueeze(0)
decoder_layer = TransformerDecoderLayer(config)
encoder_output = encoder(inputs.input_ids)
decoder_layer(inputs_embeds,encoder_output, mask, memory_mask).size()

torch.Size([1, 5, 768])

In [112]:
class TransformerDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embedding(config)
        self.decoder_layers = nn.ModuleList(
            [TransformerDecoderLayer(config) for _ in range(config.num_hidden_layers)]
        )

    def forward(self, x, encoder_output, dec_mask, memory_mask):
        x = self.embeddings(x)
        for decoder_layer in self.decoder_layers:
            x = decoder_layer(x, encoder_output, dec_mask, memory_mask)
        return x


In [113]:
decoder = TransformerDecoder(config)
decoder(inputs.input_ids, encoder_output, mask, memory_mask).size()

torch.Size([1, 5, 768])

In [114]:
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.decoder = TransformerDecoder(config)

    def forward(self, x):
        encoder_output = self.encoder(x)
        dec_mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
        memory_mask = torch.ones(seq_len, seq_len).unsqueeze(0)
        decoder_output = self.decoder(x, encoder_output, dec_mask, memory_mask)
        return decoder_output


In [116]:
transformer = Transformer(config)
transformer(inputs.input_ids).size()

torch.Size([1, 5, 768])